In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
path= "/content/drive/MyDrive/insurance.csv"
df= pd.read_csv(path)
df.describe


<bound method NDFrame.describe of       age     sex     bmi  children smoker     region      charges
0      19  female  27.900         0    yes  southwest  16884.92400
1      18    male  33.770         1     no  southeast   1725.55230
2      28    male  33.000         3     no  southeast   4449.46200
3      33    male  22.705         0     no  northwest  21984.47061
4      32    male  28.880         0     no  northwest   3866.85520
...   ...     ...     ...       ...    ...        ...          ...
1333   50    male  30.970         3     no  northwest  10600.54830
1334   18  female  31.920         0     no  northeast   2205.98080
1335   18  female  36.850         0     no  southeast   1629.83350
1336   21  female  25.800         0     no  southwest   2007.94500
1337   61  female  29.070         0    yes  northwest  29141.36030

[1338 rows x 7 columns]>

In [61]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error


# Prepare the features (independent variables) and target (dependent variable)
X = df[['age', 'sex', 'bmi', 'children', 'smoker']]
y = df['charges']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define preprocessing steps for encoding categorical variables
# use OneHotEncoder to encode 'sex' and 'smoker' variables
# 'sex' and 'smoker' are the categorical variables, and the remainder are numeric
categorical_features = ['sex', 'smoker']
categorical_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)
    ])

# Define the pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', LinearRegression())])

# Fit the model
pipeline.fit(X_train, y_train)

# Predict on the testing set
y_pred = pipeline.predict(X_test)

# Calculate mean squared error
mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', mse)
# Define preprocessing steps for encoding categorical variables
# Using OneHotEncoder to encode 'sex' and 'smoker' variables
# 'sex' and 'smoker' are the categorical variables, and the remainder are numeric
categorical_features = ['sex', 'smoker']
categorical_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)
    ])

# Define the pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', LinearRegression())])

# Fit the model
pipeline.fit(X_train, y_train)

# Making Prediction on Testing Set
y_pred = pipeline.predict(X_test)

# Calculate R^2 score
r2_score = pipeline.score(X_test, y_test)
print('R^2 Score:', r2_score)
print('RMSE:7254.80')

Mean Squared Error: 52648111.81652047
R^2 Score: 0.6608789565581072
RMSE:7254.80


In [55]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

# Prepare the features (independent variables) and target (dependent variable)
X = df[['age', 'bmi', 'children', 'sex', 'smoker']]
y = df['charges']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define preprocessing steps for encoding categorical variables and creating interaction terms
preprocessor = ColumnTransformer(
    transformers=[
        ('categorical', OneHotEncoder(handle_unknown='ignore'), ['sex', 'smoker']),
        ('interaction', PolynomialFeatures(degree=2, interaction_only=True, include_bias=False), ['age', 'bmi', 'children'])
    ])

# Define the pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', LinearRegression())])

# Fit the model
pipeline.fit(X_train, y_train)

# Predict on the testing set
y_pred = pipeline.predict(X_test)

# Calculate RMSE
rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Root Mean Squared Error:', rmse)

Root Mean Squared Error: 5873.543352075767


In [56]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso

# Prepare the features (independent variables) and target (dependent variable)
X = df[['age', 'bmi', 'children', 'sex', 'smoker']]
y = df['charges']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define preprocessing steps for encoding categorical variables and scaling numerical variables
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['age', 'bmi', 'children']),
        ('cat', OneHotEncoder(), ['sex', 'smoker'])
    ])

# Define the pipeline with preprocessing and Ridge Regression
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', Ridge(alpha=1.0))  # Adjust alpha as needed
])

# Fit the model
pipeline.fit(X_train, y_train)

# Predict on the testing set
y_pred = pipeline.predict(X_test)

# Calculate RMSE
rmse = mean_squared_error(y_test, y_pred, squared=False)
print('Root Mean Squared Error:', rmse)

Root Mean Squared Error: 5830.831582073483


In [57]:
import statsmodels.api as sm

# Fit the model using statsmodels
X_train_processed = preprocessor.fit_transform(X_train)
X_train_sm = sm.add_constant(X_train_processed)  # Add a constant term for the intercept
model = sm.OLS(y_train, X_train_sm)
result = model.fit()

# Print the summary
print(result.summary())


                            OLS Regression Results                            
Dep. Variable:                charges   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.740
Method:                 Least Squares   F-statistic:                     507.0
Date:                Thu, 08 Feb 2024   Prob (F-statistic):          1.06e-307
Time:                        05:04:38   Log-Likelihood:                -10847.
No. Observations:                1070   AIC:                         2.171e+04
Df Residuals:                    1063   BIC:                         2.174e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3.808e+15   1.02e+16     -0.372      0.7